# 2021 Repräsentationsindex
Wie gut sind Sie repräsentiert
Input:
* Gemeinde
* Jahrgang
* Geschlecht

In [35]:
import pandas as pd
import numpy as np
import cleandata as cd
import json
import os
import math
import urllib.request
from voting import apportionment

## Settings

In [36]:
export_folder = os.path.join('..', 'export', 'json')
export_folder
yearnow = 2021
marginrepresenting = 10

In [37]:
df = pd.read_csv('../data/Gemeinde_Exekutive - daten.csv')
df_age_raw = pd.read_csv('../data/KANTON_ZUERICH_bevoelkerung_1jahresklassen.csv', sep=';')

## Clean Data

In [38]:
df = cd.clean(df)

Jahrgänge nicht zugeordnet: 15
Partei nicht zugeordnet: 9
Keine Jahrgänge: 18


In [39]:
# Clean Gemeinde-Names for Join
df['Gemeinde'] = df['Gemeinde'].str.replace(' (ZH)', '', regex=False)

df_age_raw['GEMEINDE'] = df_age_raw['GEMEINDE'].str.replace('a.A.', 'am Albis', regex=False)
df_age_raw['GEMEINDE'] = df_age_raw['GEMEINDE'].str.replace('a.I.', 'am Irchel', regex=False)
df_age_raw['GEMEINDE'] = df_age_raw['GEMEINDE'].str.replace('a.d.Th.', 'an der Thur', regex=False)
df_age_raw['GEMEINDE'] = df_age_raw['GEMEINDE'].str.replace('a.S.', 'am See', regex=False)
df_age_raw['GEMEINDE'] = df_age_raw['GEMEINDE'].str.replace('a.d.L.', 'an der Limmat', regex=False)

In [40]:
# Add ID to each representativ
df['id'] = df.index + 1

In [41]:
# Prepare Gender
df_gender = df_age_raw[(df_age_raw.JAHR == 2020) & (df_age_raw.ALTERSKLASSE_CODE >= 18)]
df_gender = pd.pivot_table(df_gender, columns='GESCHLECHT', index='GEMEINDE', values='ANZAHL_PERSONEN', aggfunc='sum')

df_gender['sum'] = df_gender.sum(axis=1)
df_gender = df_gender.reset_index()

# Calc Percentage
df_gender['%m'] = round(100 / df_gender['sum'] * df_gender['Mann'])
df_gender['%w'] = round(100 / df_gender['sum'] * df_gender['Frau'])

# Prepare Alter

In [42]:
df_age = df_age_raw[df_age_raw.JAHR == 2020]

df_age = df_age.groupby(['GEMEINDE_BFS_NR', 'ALTERSKLASSE_CODE']).agg({'GEMEINDE': 'first', 'ALTERSKLASSE': 'first', 'ANZAHL_PERSONEN': 'sum'}).reset_index()

## Calculate Age Group

In [43]:
def add_age_group(x):
    if x < 18:
        return '<18'
    elif x <= 39:
        return '18-39'
    elif x <= 64:
        return '40-64'
    elif x <= 79:
        return '65-79'
    else:
        return '>80'

# Add Agegroup
df_age['agegroup'] = df_age['ALTERSKLASSE_CODE'].apply(add_age_group)
df['agegroup'] = df['Alter'].apply(add_age_group)

In [44]:
# Calculated if represented
"""
for i, row in df[df.Alter.notna()].iterrows():
    df_age.loc[df_age.ALTERSKLASSE_CODE.between(row['Alter'] - marginrepresenting, row['Alter'] + marginrepresenting) & (df_age['GEMEINDE'] == row['Gemeinde']), 'represented'] = True
    
df_age['represented'] = df_age['represented'].fillna(False)
"""

"\nfor i, row in df[df.Alter.notna()].iterrows():\n    df_age.loc[df_age.ALTERSKLASSE_CODE.between(row['Alter'] - marginrepresenting, row['Alter'] + marginrepresenting) & (df_age['GEMEINDE'] == row['Gemeinde']), 'represented'] = True\n    \ndf_age['represented'] = df_age['represented'].fillna(False)\n"

## Prepare Party

In [45]:
df_party = pd.read_csv('../data/kantonsrat2019.csv')

# Rename Wahlkreise
df_party.loc[df_party.Gemeindenamen.str.contains('Winterthur', regex=False), 'Gemeindenamen'] = 'Winterthur'
df_party.loc[df_party.Gemeindenamen.str.contains('Zürich, Kreis', regex=False), 'Gemeindenamen'] = 'Zürich'
df_party.loc[df_party.Gemeindenamen.str.contains('Ellikon a.d.Thur', regex=False), 'Gemeindenamen'] = 'Ellikon an der Thur'

# Transform
df_party['partei_c'] = df_party['Liste'].apply(cd.transform_partei)

# Group by Gemeinde
df_party = df_party.groupby(['Gemeindenamen', 'partei_c']).agg({'Stimmen': 'sum'}).reset_index()

# Calc %
df_party['2019'] = df_party.apply(lambda row: round(100 / df_party.loc[df_party.Gemeindenamen == row['Gemeindenamen'], 'Stimmen'].sum() * row['Stimmen'], 2), axis=1)

## Export JSON

In [47]:
#name = "Altikon"
def export_gemeinde(name):

    #print(name)
    
    # Select Gemeinde
    df_g = df_age[df_age.GEMEINDE == name].copy()
    df_kings = df[df.Gemeinde == name]

    # Reindex so that missing ages are filled with 0
    df_g = df_g.set_index('ALTERSKLASSE_CODE')
    df_g = df_g.reindex(np.arange(0, df_g.index.max() + 1)).fillna(0)
    df_g = df_g.reset_index()

    record = {
        "name": df_g.iloc[0]['GEMEINDE'],
        "bfs": int(df_g.iloc[0]['GEMEINDE_BFS_NR']),
        "agedistribution": [],
        "agegroups": {},
        "executives": [],
        "gender": {},
        "representativeages": [],
        "party": {},
        "executiveparty": {'is': [], 'should': []},
        "hasages": bool(df_kings['Jahrgang'].sum() > 0),
        "ageperexecutive": bool(df_kings['jahrgang_nicht_zugeordnet'].sum() == 0),
        #"partyperexecutive": bool(df_kings['partei_nicht_zugeordnet'].sum() == 0),
    }

    # Add Altersstruktur
    for i, row in df_g.iterrows():

        # Find nearest Representative
        df.loc[df.Gemeinde == name, 'yearstoagegroup'] = abs(df['Alter'] - row['ALTERSKLASSE_CODE'])

        r = {
            'age': row['ALTERSKLASSE_CODE'],
            'count': row['ANZAHL_PERSONEN'],
            'yeartorepresentative': min(df.loc[df.Gemeinde == name, 'yearstoagegroup'])
        }
        if math.isnan(r['yeartorepresentative']):
           r['yeartorepresentative'] = None
                      
        record['agedistribution'].append(r)

    # Add Räte Version 1    
    for i, row in df[df.Gemeinde == name].sort_values(['Jahrgang', 'Name'], ascending=[False, True]).iterrows():

        if math.isnan(row['Jahrgang']):
            jg = None
        else:
            jg = round(row['Jahrgang']) if row['jahrgang_nicht_zugeordnet'] == False else None

        r = {
            "id": row['id'],
            "name": row['Name_cleaned'],
            "agegroup": jg,
            "agecalculated": row['Alter'] if (row['jahrgang_nicht_zugeordnet'] == False and math.isnan(row['Alter']) == False) else None,
            "party": row['Partei'] if row['partei_nicht_zugeordnet'] == False else None,
            "sex": row['Geschlecht']
        }
        record['executives'].append(r)

    # Calculate Age Group Percentage per Gemeinde
    df_agegroup = df_g[df_g.agegroup != '<18'].groupby('agegroup').agg({'ANZAHL_PERSONEN': 'sum'}).reset_index()
    df_agegroup['value'] = round(100 / df_agegroup['ANZAHL_PERSONEN'].sum() * df_agegroup['ANZAHL_PERSONEN'])

    for i, row in df_agegroup.iterrows():

        record['agegroups'][row['agegroup']] = {
            'inhabitants': row['value'],
            'executives': round(100 / len(df_kings) * len(df_kings[df_kings.agegroup == row['agegroup']]))
        }

    # Add representative age groups
    df_sub = df[df.Gemeinde == name].groupby(['Jahrgang', 'Name_cleaned']).agg({'id': 'first', 'jahrgang_nicht_zugeordnet': 'first'})
    for i, rows in df_sub.groupby(level=0):

        ids = []
        for j, rowj in df_sub.iloc[df_sub.index.get_level_values('Jahrgang') == i].reset_index().iterrows():
            if(rowj['jahrgang_nicht_zugeordnet'] == False):
                ids.append(rowj['id'])  

        record['representativeages'].append({
            'age': round(yearnow - i),
            'ids': ids})

    # Add Gender
    record['gender']['m'] = int(df_gender.loc[df_gender.GEMEINDE == name, '%m'])
    record['gender']['w'] = int(df_gender.loc[df_gender.GEMEINDE == name, '%w'])

    # Add Party
    df_p_2019 = df_party[df_party.Gemeindenamen == name].copy().reset_index()

    # Calculate Percentage of Party in Board
    df_p_board = df_kings.groupby('partei_c').agg({'Name': 'count'}).reset_index()
    df_p_board['board'] = round(100 / df_p_board['Name'].sum() * df_p_board['Name'])

    df_p_board = df_p_board.merge(df_p_2019, how = 'outer', on = 'partei_c')
    df_p_board['2019'] = df_p_board['2019'].fillna(0)
    df_p_board['board'] = df_p_board['board'].fillna(0)

    # Order by max
    df_p_board['max'] = df_p_board.apply(lambda row: row['board'] if row['board'] >= row['2019'] else row['2019'], axis=1)
    df_p_board['diff'] = abs(df_p_board['2019'] - df_p_board['board'])
    df_p_board = df_p_board.sort_values('diff', ascending=False)

    # Remove less 5 percent
    df_p_board = df_p_board[df_p_board['max'] >= 5]

    r = {}
    for i, row in df_p_board.iterrows():
        r[row['partei_c']] = {
            '2019': round(row['2019']),
            'board': row['board']
        }
    record['party'] = r

    # Calculate Executive Party IS
    for i, row in df_kings.groupby('partei_c').count().sort_values('Gemeinde', ascending=False).reset_index().iterrows():
        record['executiveparty']['is'] += [row['partei_c']] * row['Gemeinde']

    # Calculate Executive Party SHOULD
    df_p_2019['should'] = apportionment.hagenbach_bischoff(list(df_p_2019['2019']), len(df_kings))
    for i, row in df_p_2019[df_p_2019.should > 0].sort_values('should', ascending=False).iterrows():
        record['executiveparty']['should'] += [row['partei_c']] * row['should']

    # Store
    #with open(os.path.join(export_folder, '230.json'), "w") as f:
    with open(os.path.join(export_folder, '%s.json' % int(df_g.iloc[0]['GEMEINDE_BFS_NR'])), "w") as f:
        json.dump(record, f)


#export_gemeinde('Winterthur')



In [48]:
df_gemeinden = df.groupby('Gemeinde').count().reset_index()
    
for i, row in df_gemeinden.iterrows():
    export_gemeinde(row['Gemeinde'])
    
print("Finito")

Finito


## Tests

In [ ]:
# Settings
gemeinde = 'Dürnten'
alter = 32

my_agegroup = add_age_group(alter)

### Representation Test mit fixen Altersgruppe
Gruppen:
* < 18
* 18 - 39
* 40 - 64
* 65 - 79
+ 80+

In [ ]:
# Select Gemeinde in Gemeinderäte
df_raete = df[df.Gemeinde == gemeinde].copy()
df_raete['agegroup'] = df_raete['Alter'].apply(add_age_group)

# Group by agegroup and calc Percentage
df_test = df_age[df_age.GEMEINDE == gemeinde].groupby('agegroup').agg({'ANZAHL_PERSONEN': 'sum'})
df_test['%'] = round(100 / df_test['ANZAHL_PERSONEN'].sum() * df_test['ANZAHL_PERSONEN'])
df_test['personen_soll'] = len(df_raete) / 100 * df_test['%']

# Group raete
df_raete = df_raete.groupby('agegroup').agg({'Name': 'count'}).rename(columns={'Name': 'personen_ist'})

# Join
df_test = df_test.join(df_raete)
df_test = df_test.fillna(0)
df_test['abbr'] = df_test['personen_ist'] - df_test['personen_soll']

df_test

## Representations Test mit fluiden Altersgruppen

In [ ]:
agegroup_plus_minus = 5

df_test = df_age[df_age.GEMEINDE == gemeinde].copy()

# Select your age
df_yourage = df_test[(df_test.ALTERSKLASSE_CODE.between(alter - agegroup_plus_minus, alter + agegroup_plus_minus))]

# Calc percentage of your age
round(100 / df_test['ANZAHL_PERSONEN'].sum() * df_yourage['ANZAHL_PERSONEN'].sum())

# Group raete
df_raete = df[df.Gemeinde == gemeinde].copy()
#df_raete = df_raete.groupby('Alter').agg({'Name': 'count'}).rename(columns={'Name': 'count'})

#df_test = df_test.set_index('ALTERSKLASSE_CODE')

# Loop through gemeinderäte
for i, row in df_raete.iterrows():
    df_test.loc[df_test.ALTERSKLASSE_CODE.between(row['Alter'] - 5, row['Alter'] + 5), 'represented'] = True
    
df_test['represented'] = df_test['represented'].fillna(False)

# Join
#df_test = df_test.join(df_raete)

In [ ]:
df_test = df_age[df_age.GEMEINDE == 'Winterthur']

# Set Color
df_test['color'] = df_test['represented'].apply(lambda x: 'red' if x else 'grey')

#df_plot = df_test.set_index('ALTERSKLASSE_CODE')
df_test.plot('ALTERSKLASSE_CODE', 'ANZAHL_PERSONEN', kind='bar', figsize=(10, 10), color=df_test['color'].tolist())